In [1]:
from transformers import AutoModel, AutoTokenizer, LongformerSelfAttention
import torch

import copy

2023-03-27 13:20:23.236224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 13:20:23.353124: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 13:20:23.353156: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-27 13:20:24.251696: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def sbert_to_longformer(base_model, max_pos=4096, attention_window=512):
    """Transform a `base_model` (RoBERTa) into a Longformer with sparce attention."""
    
    logger.info("Transform to longformer")
    model = base_model._modules["0"]._modules["auto_model"]
    config = model.config
    tokenizer = base_model.tokenizer
    
    tokenizer.model_max_length = max_pos
    tokenizer.init_kwargs["model_max_length"] = max_pos
    # tokenizer._tokenizer.truncation["max_length"] = attention_window

    current_max_pos, embed_size = model.embeddings.position_embeddings.weight.shape
    max_pos += 2  # NOTE: RoBERTa has positions 0,1 reserved, so embedding size is max position + 2
    config.max_position_embeddings = max_pos

    assert max_pos > current_max_pos
    # allocate a larger position embedding matrix
    new_pos_embed = model.embeddings.position_embeddings.weight.new_empty(
        max_pos, embed_size
    )
    # copy position embeddings over and over to initialize the new position embeddings

    k = 2
    step = current_max_pos - 2
    while k < max_pos - 1:
        new_pos_embed[k : (k + step)] = model.embeddings.position_embeddings.weight[2:]
        k += step

    model.embeddings.position_embeddings.weight.data = new_pos_embed
    model.embeddings.position_ids.data = torch.tensor(
        [i for i in range(max_pos)]
    ).reshape(1, max_pos)

    # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
    config.attention_window = [attention_window] * config.num_hidden_layers

    for i, layer in enumerate(model.encoder.layer):
        longformer_self_attn = LongformerSelfAttention(config, layer_id=i)
        longformer_self_attn.query = layer.attention.self.query
        longformer_self_attn.key = layer.attention.self.key
        longformer_self_attn.value = layer.attention.self.value

        longformer_self_attn.query_global = copy.deepcopy(layer.attention.self.query)
        longformer_self_attn.key_global = copy.deepcopy(layer.attention.self.key)
        longformer_self_attn.value_global = copy.deepcopy(layer.attention.self.value)

        layer.attention.self = longformer_self_attn
    
    return model, tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [4]:
type(model)

transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaModel

In [7]:
model.modules

<bound method Module.modules of XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0): XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05

In [4]:
model, tokenizer = to_longformer(model, tokenizer)

In [13]:
max_pos

NameError: name 'max_pos' is not defined

In [5]:
model.embeddings.position_embeddings.weight.shape

torch.Size([4098, 768])

In [6]:
model.modules

<bound method Module.modules of XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0): XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bia

In [7]:
save_model_to = "/home/helder/doc/mecd/thesis/models/longformer-test"

model.save_pretrained(save_model_to)
tokenizer.save_pretrained(save_model_to)

('/home/helder/doc/mecd/thesis/models/longformer-test/tokenizer_config.json',
 '/home/helder/doc/mecd/thesis/models/longformer-test/special_tokens_map.json',
 '/home/helder/doc/mecd/thesis/models/longformer-test/sentencepiece.bpe.model',
 '/home/helder/doc/mecd/thesis/models/longformer-test/added_tokens.json',
 '/home/helder/doc/mecd/thesis/models/longformer-test/tokenizer.json')

In [ ]:
# Load Lonformer by SentenceTransformer

embedding_model = SentenceTransformer(model_path)



embedding_model._modules["0"]._modules[
            "auto_model"
    ] = LongformerModel.from_pretrained(model_path)
callable_model = SentenceTransformerBackend(embedding_model)

# Sentence Transformer to Longmodel

In [ ]:
embedding_model = SentenceTransformer(model_path)



# STS Benchmark


In [5]:
import pandas as pd

In [8]:
results_original = pd.read_csv("/home/helder/doc/mecd/thesis/outputs/similarity_evaluation_sts-test_resultsi_original.csv")
results_original

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.854345,0.868222,0.86476,0.863053,0.863467,0.861315,0.827153,0.822673


In [9]:
results_test = pd.read_csv("/home/helder/doc/mecd/thesis/outputs/similarity_evaluation_sts-test_results.csv")
results_test

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.854345,0.868222,0.86476,0.863053,0.863467,0.861315,0.827153,0.822673


In [10]:
results_test = pd.read_csv("/home/helder/doc/mecd/thesis/outputs/similarity_evaluation_sts-test_resultsi_longformer.csv")
results_test

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.854345,0.868222,0.86476,0.863053,0.863467,0.861315,0.827153,0.822673
